# Get combined mechanisms data

In [1]:
import numpy, os
import pandas as pd

## Read the data

In [2]:
MODEL_NAME = "llama2_13B" # "llama2_7B" # "gpt2-xl"

In [3]:
datasets = {"gpt2-xl": [{"name": "exact_recall",
             "queries_file": "/cephyr/users/lovhag/Alvis/projects/rome/data/eval_on_fact_recall_set/gpt2-xl/1000_exact.json",
             "CT_results_dir": "/cephyr/users/lovhag/Alvis/projects/rome/data/eval_on_fact_recall_set/gpt2-xl/causal_trace_2377617/cases",
             "filename_template": "knowledge_{}_mlp.npz" 
            },
              {"name": "prompt_bias",
              "queries_file": "/cephyr/users/lovhag/Alvis/projects/fact-recall-detection/data/CT_results/gpt2_xl/synthetic_data/prompt_bias_bias.jsonl",
              "CT_results_dir": "/cephyr/users/lovhag/Alvis/projects/fact-recall-detection/data/CT_results/gpt2_xl/synthetic_data/prompt_bias_bias/cases",
              "filename_template": "{}_candidate_mlp.npz"
              },
              {"name": "person_name_bias",
              "queries_file": "/cephyr/users/lovhag/Alvis/projects/fact-recall-detection/data/CT_results/gpt2_xl/synthetic_data/person_name_bias.jsonl",
              "CT_results_dir": "/cephyr/users/lovhag/Alvis/projects/fact-recall-detection/data/CT_results/gpt2_xl/synthetic_data/person_name_bias/cases",
              "filename_template": "{}_candidate_mlp.npz"
              },
              {"name": "guesswork",
              "queries_file": "/cephyr/users/lovhag/Alvis/projects/rome/data/eval_on_fact_recall_set/gpt2-xl/1000_guesswork.json",
              "CT_results_dir": "/cephyr/users/lovhag/Alvis/projects/rome/data/eval_on_fact_recall_set/gpt2-xl/causal_trace_guesswork_2388522/cases",
              "filename_template": "knowledge_{}_mlp.npz"
              }
             ],
            "llama2_7B": [{"name": "exact_recall",
             "queries_file": "/cephyr/users/lovhag/Alvis/projects/fact-recall-detection/data/CT_sensitivity_recall_eval_sets/llama2_7B/1000_exact.jsonl",
             "CT_results_dir": "/cephyr/users/lovhag/Alvis/projects/rome/data/eval_on_fact_recall_set/llama2_7B/causal_trace_exact_2398046/cases",
             "filename_template": "{}_candidate_mlp.npz" 
            },
              {"name": "prompt_bias",
              "queries_file": "/cephyr/users/lovhag/Alvis/projects/fact-recall-detection/data/CT_results/llama2_7B/synthetic_data/prompt_bias_bias.jsonl",
              "CT_results_dir": "/cephyr/users/lovhag/Alvis/projects/fact-recall-detection/data/CT_results/llama2_7B/synthetic_data/prompt_bias_bias/cases",
              "filename_template": "{}_candidate_mlp.npz"
              },
              {"name": "person_name_bias",
              "queries_file": "/cephyr/users/lovhag/Alvis/projects/fact-recall-detection/data/CT_results/llama2_7B/synthetic_data/person_name_bias.jsonl",
              "CT_results_dir": "/cephyr/users/lovhag/Alvis/projects/fact-recall-detection/data/CT_results/llama2_7B/synthetic_data/person_name_bias/cases",
              "filename_template": "{}_candidate_mlp.npz"
              },
              {"name": "guesswork",
              "queries_file": "/cephyr/users/lovhag/Alvis/projects/fact-recall-detection/data/CT_sensitivity_recall_eval_sets/llama2_7B/1000_guesswork.jsonl",
              "CT_results_dir": "/cephyr/users/lovhag/Alvis/projects/rome/data/eval_on_fact_recall_set/llama2_7B/causal_trace_guesswork_2398048/cases",
              "filename_template": "{}_candidate_mlp.npz"
              }
             ],
            "llama2_13B": [{"name": "exact_recall",
             "queries_file": "/cephyr/users/lovhag/Alvis/projects/fact-recall-detection/data/CT_sensitivity_recall_eval_sets/llama2_13B/1000_exact.jsonl",
             "CT_results_dir": "/cephyr/users/lovhag/Alvis/projects/rome/data/eval_on_fact_recall_set/llama2_13B/causal_trace_exact_2904371/cases",
             "filename_template": "{}_candidate_mlp.npz" 
            },
              {"name": "prompt_bias",
              "queries_file": "/cephyr/users/lovhag/Alvis/projects/fact-recall-detection/data/CT_results/llama2_13B/synthetic_data/prompt_bias_bias.jsonl",
              "CT_results_dir": "/cephyr/users/lovhag/Alvis/projects/fact-recall-detection/data/CT_results/llama2_13B/synthetic_data/prompt_bias_bias/cases",
              "filename_template": "{}_candidate_mlp.npz"
              },
              {"name": "person_name_bias",
              "queries_file": "/cephyr/users/lovhag/Alvis/projects/fact-recall-detection/data/CT_results/llama2_13B/synthetic_data/person_name_bias.jsonl",
              "CT_results_dir": "/cephyr/users/lovhag/Alvis/projects/fact-recall-detection/data/CT_results/llama2_13B/synthetic_data/person_name_bias/cases",
              "filename_template": "{}_candidate_mlp.npz"
              },
              {"name": "guesswork",
              "queries_file": "/cephyr/users/lovhag/Alvis/projects/fact-recall-detection/data/CT_sensitivity_recall_eval_sets/llama2_13B/1000_guesswork.jsonl",
              "CT_results_dir": "/cephyr/users/lovhag/Alvis/projects/rome/data/eval_on_fact_recall_set/llama2_13B/causal_trace_guesswork_2904686/cases",
              "filename_template": "{}_candidate_mlp.npz"
              }
             ],
            }

data_shares = {"exact_recall": 0.33,
               "prompt_bias": 0.17,
               "person_name_bias": 0.17,
               "guesswork": 0.33
              }
assert sum(data_shares.values())==1

num_samples = 1000

In [4]:
data = pd.DataFrame()
for dataset in datasets[MODEL_NAME]:
    tmp_data = pd.read_json(dataset["queries_file"], lines=dataset["queries_file"].endswith("jsonl")).sample(
        int(num_samples*data_shares[dataset["name"]]), random_state=42)
    tmp_data["type"] = dataset["name"]
    tmp_data["CT_results_dir"] = dataset["CT_results_dir"]
    tmp_data["filename_template"] = dataset["filename_template"]
    data = pd.concat((data, tmp_data), ignore_index=True)
    
data

,obj_label,sub_label,predicate_id,source,prompt,template,sub_view_rates,obj_view_rates,answers,p_answers,...,answers_for_PB_She,confidence,candidate_p,top10_tokens,prediction_p,prediction,confident_flag,answers_for_PB_It,answers_for_PB_The_city,answers_for_PB_The_organisation
0,Albania,Tirana,P1376,TREx_UHN,"Tirana, that is the capital city of","[X], that is the capital city of [Y]",28997.666667,199427.583333,Alban,0.886154,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Italy,Orzo,P495,TREx_UHN,Orzo was created in,[X] was created in [Y],11064.500000,338858.666667,Italy,0.215916,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Boston,Powerman 5000,P740,TREx_UHN,"Powerman 5000, created in","[X], created in [Y]",17944.583333,161046.000000,Boston,0.022921,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Taiwan,Taipei,P1376,TREx_UHN,Taipei is the capital of,[X] is the capital of [Y],58937.833333,376408.916667,Taiwan,0.75268,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Japan,Azumanga Daioh,P495,TREx_UHN,"Azumanga Daioh, that was created in","[X], that was created in [Y]",7330.833333,491464.833333,Japan,0.026744,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Canada,The Midway State,P495,TREx_UHN,The Midway State was from,[X] was from [Y],388.916667,598412.500000,Brook,0.074563,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
996,theology,Friedrich Spanheim,P101,TREx_UHN,Friedrich Spanheim works in the field of,[X] works in the field of [Y],204.416667,36685.666667,philosophy,0.017109,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
997,Luxembourg,Jean Muller,P19,Google_RE_UHN,Jean Muller was originally from,[X] was originally from [Y],199.583333,204737.166667,France,0.05725,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
998,Cork,Samuel Forde,P19,Google_RE_UHN,Samuel Forde was born in,[X] was born in [Y],56.250000,4286.416667,London,0.033935,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
data.type.value_counts()

exact_recall        330
guesswork           330
prompt_bias         170
person_name_bias    170
Name: type, dtype: int64

Save the data

In [6]:
data.to_json(f"/cephyr/users/lovhag/Alvis/projects/rome/data/eval_on_fact_recall_set/{MODEL_NAME}/1000_combined_mechanisms.json", orient="records")